In [14]:
import scipy as cp
import sklearn
import spacy


In [15]:
en_nlp = spacy.load('en')

In [17]:
import post_collection as p
cleanjokes = p.collect_posts('cleanjokes', 50)
cleanjokes += p.collect_posts('ShortCleanFunny', 50)

In [18]:
len(cleanjokes)

2726

In [15]:
nsfwjokes = p.collect_posts('dirtyjokes', 50)
nsfwjokes += p.collect_posts('meanjokes', 50)

In [16]:
len(nsfwjokes)

13340

In [4]:
nsfwjokes[:10]

[Submission(id='5xf39a'),
 Submission(id='5wwvgb'),
 Submission(id='5wqm6q'),
 Submission(id='5wo5lw'),
 Submission(id='5wh498'),
 Submission(id='5w42m0'),
 Submission(id='5vt5gz'),
 Submission(id='5vn8it'),
 Submission(id='5vd689'),
 Submission(id='5v8d1c')]

In [5]:
cleanjokes[:10]

[Submission(id='5xqgbm'),
 Submission(id='5xqft2'),
 Submission(id='5xqf3l'),
 Submission(id='5xjt6b'),
 Submission(id='5xga57'),
 Submission(id='5xd9qc'),
 Submission(id='5xd1zm'),
 Submission(id='5xcvnq'),
 Submission(id='5x87ev'),
 Submission(id='5x63kn')]

In [6]:
def extract_features(post):
    feats = {}
    feats['joketext'] = post.title + post.selftext
    
    #Positive class is if the joke is inappropriate
    feats['label'] = True if post.domain.split('.')[1].lower() in {'dirtyjokes', 'meanjokes'} else False
    return feats

In [9]:
extract_features(nsfwjokes[2])

{'joketext': u"What's the worst part about getting locked out of your car outside of an abortion clinic?Having to go inside to ask for a coat hanger",
 'label': True}

In [10]:
extract_features(cleanjokes[5])

{'joketext': u'Why are karate instructors the worst party guests?They always show up empty handed.',
 'label': False}

In [17]:
nsfwtrain = nsfwjokes[:(len(cleanjokes) + 1)]

In [19]:
trjokes = cleanjokes[:1600] + nsfwtrain[:1600]
valjokes = cleanjokes[1600:2100] + nsfwtrain[1600:2100]
tstjokes = cleanjokes[2100:] + nsfwtrain[2100:]

In [31]:
en_nlp.pipeline = [en_nlp.tagger]
def get_train_sents(posts):
    textiter = []
    for post in posts:
        txt = extract_features(post)['joketext']
        lemmatized = ' '.join([token.lemma_ for token in en_nlp(txt)])
        textiter.append(lemmatized)
    return textiter

In [35]:
trsents = get_train_sents(trjokes)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfi = TfidfVectorizer(#...
    min_df=10,
    max_df=0.8,
    max_features=1000,
    stop_words='english',
    norm='l2'
                            )

In [37]:
trainX = tfi.fit_transform(trsents)

In [38]:
trainX

<3200x757 sparse matrix of type '<type 'numpy.float64'>'
	with 24272 stored elements in Compressed Sparse Row format>

In [40]:
trainy = [extract_features(joke)['label'] for joke in trjokes]

In [54]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [55]:
svc = LinearSVC()
knn = KNeighborsClassifier()
rfc = RandomForestClassifier()

In [59]:
svc.fit(trainX, trainy)
knn.fit(trainX, trainy)
rfc.fit(trainX, trainy)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [57]:
valX = tfi.transform(get_train_sents(valjokes))

In [66]:
valy = [extract_features(joke)['label'] for joke in valjokes]

In [48]:
svcpred = svc.predict(valX)

In [60]:
knnpred = knn.predict(valX)

In [61]:
rfcpred = rfc.predict(valX)

In [52]:
from sklearn.metrics import accuracy_score

In [63]:
accuracy_score(svcpred, valy)

0.76200000000000001

In [64]:
accuracy_score(knnpred, valy)

0.61899999999999999

In [67]:
accuracy_score(rfcpred, valy)

0.749